# メタクラスと属性の基本

このノートブックでは、メタクラスと属性の基本的な操作とベストプラクティスを学びます。


In [ ]:
# __new__でオブジェクトの構築を制御する

# シングルトンパターンの実装
class Singleton:
    """シングルトンクラス"""
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance
    
    def __init__(self):
        if not hasattr(self, 'initialized'):
            self.initialized = True
            self.value = 0
    
    def set_value(self, value):
        """値を設定"""
        self.value = value
    
    def get_value(self):
        """値を取得"""
        return self.value

# 使用例
print("=== シングルトンパターン ===")
singleton1 = Singleton()
singleton1.set_value(42)

singleton2 = Singleton()
print(f"Singleton 1: {singleton1.get_value()}")
print(f"Singleton 2: {singleton2.get_value()}")
print(f"同じインスタンス: {singleton1 is singleton2}")

# 設定クラスでのシングルトン
class Config:
    """設定クラス（シングルトン）"""
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance
    
    def __init__(self):
        if not hasattr(self, 'initialized'):
            self.initialized = True
            self.settings = {}
    
    def set_setting(self, key, value):
        """設定を追加"""
        self.settings[key] = value
    
    def get_setting(self, key, default=None):
        """設定を取得"""
        return self.settings.get(key, default)

# 使用例
config1 = Config()
config1.set_setting('debug', True)
config1.set_setting('log_level', 'INFO')

config2 = Config()
print(f"\n=== 設定クラス ===")
print(f"Config 1 debug: {config1.get_setting('debug')}")
print(f"Config 2 debug: {config2.get_setting('debug')}")
print(f"同じインスタンス: {config1 is config2}")

# 不変オブジェクトの作成
class ImmutablePoint:
    """不変な点クラス"""
    
    def __new__(cls, x, y):
        # 既存のインスタンスをチェック
        for instance in cls._instances:
            if instance.x == x and instance.y == y:
                return instance
        
        # 新しいインスタンスを作成
        instance = super().__new__(cls)
        instance._x = x
        instance._y = y
        cls._instances.add(instance)
        return instance
    
    def __init__(self, x, y):
        # 既に初期化済みの場合はスキップ
        if hasattr(self, '_initialized'):
            return
        self._initialized = True
    
    @property
    def x(self):
        return self._x
    
    @property
    def y(self):
        return self._y
    
    def __repr__(self):
        return f"ImmutablePoint({self.x}, {self.y})"
    
    def __hash__(self):
        return hash((self.x, self.y))
    
    def __eq__(self, other):
        return isinstance(other, ImmutablePoint) and self.x == other.x and self.y == other.y

# クラス変数を初期化
ImmutablePoint._instances = set()

# 使用例
print("\n=== 不変オブジェクト ===")
point1 = ImmutablePoint(1, 2)
point2 = ImmutablePoint(1, 2)
point3 = ImmutablePoint(3, 4)

print(f"Point 1: {point1}")
print(f"Point 2: {point2}")
print(f"Point 3: {point3}")
print(f"Point 1 is Point 2: {point1 is point2}")
print(f"Point 1 == Point 2: {point1 == point2}")
print(f"Point 1 is Point 3: {point1 is point3}")


In [ ]:
# __init_subclass__でサブクラスを検証する

# サブクラスの検証
class BaseModel:
    """ベースモデルクラス"""
    
    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        
        # 必須メソッドの存在をチェック
        required_methods = ['validate', 'save']
        for method in required_methods:
            if not hasattr(cls, method):
                raise TypeError(f"{cls.__name__} must implement {method} method")
        
        # 必須属性の存在をチェック
        if not hasattr(cls, 'table_name'):
            raise TypeError(f"{cls.__name__} must define table_name attribute")
        
        print(f"サブクラス {cls.__name__} が正常に作成されました")

class User(BaseModel):
    """ユーザークラス"""
    table_name = 'users'
    
    def __init__(self, name, email):
        self.name = name
        self.email = email
    
    def validate(self):
        """バリデーション"""
        if not self.name:
            raise ValueError("名前は必須です")
        if not self.email or '@' not in self.email:
            raise ValueError("有効なメールアドレスを入力してください")
        return True
    
    def save(self):
        """保存"""
        if self.validate():
            print(f"ユーザー {self.name} を保存しました")
            return True
        return False

# 使用例
print("=== サブクラスの検証 ===")
user = User("Alice", "alice@example.com")
user.save()

# エラーケース
try:
    class InvalidModel(BaseModel):
        pass
except TypeError as e:
    print(f"エラー: {e}")

# 設定の自動適用
class ConfigurableBase:
    """設定可能なベースクラス"""
    
    def __init_subclass__(cls, config=None, **kwargs):
        super().__init_subclass__(**kwargs)
        
        if config:
            # 設定をクラス属性に適用
            for key, value in config.items():
                setattr(cls, key, value)
        
        # デフォルト設定を適用
        if not hasattr(cls, 'debug'):
            cls.debug = False
        if not hasattr(cls, 'log_level'):
            cls.log_level = 'INFO'
        
        print(f"クラス {cls.__name__} に設定を適用しました")

class APIClient(ConfigurableBase, config={'timeout': 30, 'retries': 3}):
    """APIクライアントクラス"""
    
    def __init__(self, base_url):
        self.base_url = base_url
    
    def get_config(self):
        """設定を取得"""
        return {
            'timeout': self.timeout,
            'retries': self.retries,
            'debug': self.debug,
            'log_level': self.log_level
        }

class DatabaseClient(ConfigurableBase, config={'pool_size': 10, 'timeout': 60}):
    """データベースクライアントクラス"""
    
    def __init__(self, connection_string):
        self.connection_string = connection_string
    
    def get_config(self):
        """設定を取得"""
        return {
            'pool_size': self.pool_size,
            'timeout': self.timeout,
            'debug': self.debug,
            'log_level': self.log_level
        }

# 使用例
print("\n=== 設定の自動適用 ===")
api_client = APIClient("https://api.example.com")
print(f"API設定: {api_client.get_config()}")

db_client = DatabaseClient("postgresql://localhost:5432/mydb")
print(f"DB設定: {db_client.get_config()}")

# プラグインシステム
class PluginBase:
    """プラグインベースクラス"""
    _plugins = {}
    
    def __init_subclass__(cls, plugin_name=None, **kwargs):
        super().__init_subclass__(**kwargs)
        
        # プラグイン名を設定
        if plugin_name:
            cls.plugin_name = plugin_name
        else:
            cls.plugin_name = cls.__name__.lower()
        
        # プラグインを登録
        cls._plugins[cls.plugin_name] = cls
        
        # 必須メソッドの存在をチェック
        if not hasattr(cls, 'execute'):
            raise TypeError(f"Plugin {cls.plugin_name} must implement execute method")
        
        print(f"プラグイン {cls.plugin_name} を登録しました")
    
    @classmethod
    def get_plugin(cls, name):
        """プラグインを取得"""
        return cls._plugins.get(name)
    
    @classmethod
    def list_plugins(cls):
        """プラグイン一覧を取得"""
        return list(cls._plugins.keys())

class EmailPlugin(PluginBase, plugin_name='email'):
    """メールプラグイン"""
    
    def execute(self, data):
        """実行"""
        print(f"メール送信: {data}")

class SMSSPlugin(PluginBase, plugin_name='sms'):
    """SMSプラグイン"""
    
    def execute(self, data):
        """実行"""
        print(f"SMS送信: {data}")

class NotificationPlugin(PluginBase, plugin_name='notification'):
    """通知プラグイン"""
    
    def execute(self, data):
        """実行"""
        print(f"通知送信: {data}")

# 使用例
print("\n=== プラグインシステム ===")
print(f"登録されたプラグイン: {PluginBase.list_plugins()}")

# プラグインを取得して実行
email_plugin = PluginBase.get_plugin('email')
if email_plugin:
    email_plugin().execute("Hello World")

sms_plugin = PluginBase.get_plugin('sms')
if sms_plugin:
    sms_plugin().execute("Hello World")


In [ ]:
# __slots__でメモリ使用量を削減する

import sys

# 通常のクラス
class RegularClass:
    """通常のクラス"""
    def __init__(self, name, age, email):
        self.name = name
        self.age = age
        self.email = email

# __slots__を使用したクラス
class SlotsClass:
    """__slots__を使用したクラス"""
    __slots__ = ['name', 'age', 'email']
    
    def __init__(self, name, age, email):
        self.name = name
        self.age = age
        self.email = email

# メモリ使用量の比較
print("=== メモリ使用量の比較 ===")

# 通常のクラス
regular_obj = RegularClass("Alice", 25, "alice@example.com")
print(f"通常のクラスのメモリ使用量: {sys.getsizeof(regular_obj)} bytes")

# __slots__を使用したクラス
slots_obj = SlotsClass("Alice", 25, "alice@example.com")
print(f"__slots__クラスのメモリ使用量: {sys.getsizeof(slots_obj)} bytes")

# 複数のインスタンスでの比較
regular_objects = [RegularClass(f"User{i}", 20+i, f"user{i}@example.com") for i in range(1000)]
slots_objects = [SlotsClass(f"User{i}", 20+i, f"user{i}@example.com") for i in range(1000)]

regular_memory = sum(sys.getsizeof(obj) for obj in regular_objects)
slots_memory = sum(sys.getsizeof(obj) for obj in slots_objects)

print(f"\n1000個のインスタンスでの比較:")
print(f"通常のクラス: {regular_memory} bytes")
print(f"__slots__クラス: {slots_memory} bytes")
print(f"メモリ削減: {regular_memory - slots_memory} bytes ({(regular_memory - slots_memory) / regular_memory * 100:.1f}%)")

# データベースモデルでの__slots__活用
class UserModel:
    """ユーザーモデル（__slots__使用）"""
    __slots__ = ['user_id', 'name', 'email', 'created_at', 'is_active']
    
    def __init__(self, user_id, name, email, created_at=None, is_active=True):
        self.user_id = user_id
        self.name = name
        self.email = email
        self.created_at = created_at or '2023-01-01'
        self.is_active = is_active
    
    def get_info(self):
        """情報を取得"""
        return {
            'user_id': self.user_id,
            'name': self.name,
            'email': self.email,
            'created_at': self.created_at,
            'is_active': self.is_active
        }

class ProductModel:
    """商品モデル（__slots__使用）"""
    __slots__ = ['product_id', 'name', 'price', 'category', 'in_stock']
    
    def __init__(self, product_id, name, price, category, in_stock=True):
        self.product_id = product_id
        self.name = name
        self.price = price
        self.category = category
        self.in_stock = in_stock
    
    def get_info(self):
        """情報を取得"""
        return {
            'product_id': self.product_id,
            'name': self.name,
            'price': self.price,
            'category': self.category,
            'in_stock': self.in_stock
        }

# 使用例
print("\n=== データベースモデルでの__slots__活用 ===")
user = UserModel(1, "Alice", "alice@example.com")
product = ProductModel(1, "Laptop", 100000, "Electronics")

print(f"ユーザー情報: {user.get_info()}")
print(f"商品情報: {product.get_info()}")

# メモリ使用量
print(f"ユーザーモデルのメモリ使用量: {sys.getsizeof(user)} bytes")
print(f"商品モデルのメモリ使用量: {sys.getsizeof(product)} bytes")

# エラーケース（__slots__で定義されていない属性）
try:
    user.phone = "090-1234-5678"  # これはエラーになる
except AttributeError as e:
    print(f"エラー: {e}")

# プロパティでの__slots__活用
class ConfigModel:
    """設定モデル（__slots__とプロパティ使用）"""
    __slots__ = ['_config', '_initialized']
    
    def __init__(self, config=None):
        self._config = config or {}
        self._initialized = True
    
    @property
    def debug(self):
        """デバッグモード"""
        return self._config.get('debug', False)
    
    @property
    def log_level(self):
        """ログレベル"""
        return self._config.get('log_level', 'INFO')
    
    @property
    def timeout(self):
        """タイムアウト"""
        return self._config.get('timeout', 30)
    
    def get_config(self):
        """設定を取得"""
        return self._config.copy()
    
    def set_config(self, key, value):
        """設定を設定"""
        self._config[key] = value

# 使用例
print("\n=== プロパティでの__slots__活用 ===")
config = ConfigModel({'debug': True, 'log_level': 'DEBUG', 'timeout': 60})
print(f"デバッグモード: {config.debug}")
print(f"ログレベル: {config.log_level}")
print(f"タイムアウト: {config.timeout}")
print(f"設定: {config.get_config()}")

# メモリ使用量
print(f"設定モデルのメモリ使用量: {sys.getsizeof(config)} bytes")
